In [3]:
!pip install lenskit 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 4.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.40.1
    Uninstalling llvmlite-0.40.1:
      Successfully uninstalled llvmlite-0.40.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
  Attempting uninstall: numba
    Found existing installation: numba 0.57.1
    Uninstalling numba-0.57.1:
      Successfully uninstalled numba-0.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [2]:
import pandas as pd

from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms.item_knn import ItemItem

from sklearn.model_selection import train_test_split, KFold

import numpy as np
import statistics

import pickle

In [3]:
data_folder = 'data/'

item_ratings_df = pd.read_csv(data_folder + 'user_ratings.csv')
# item_ratings_df = pd.read_csv(data_folder + 'user_ratings_600k.csv')
# item_ratings_df = pd.read_csv(data_folder + 'user_ratings_1.8m.csv')
item_info = pd.read_csv(data_folder + 'games.csv')
item_mecahnics = pd.read_csv(data_folder + 'mechanics.csv')
item_subcategories = pd.read_csv(data_folder + 'subcategories.csv')
item_themes = pd.read_csv(data_folder + 'themes.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/user_ratings.csv'

In [ ]:
# item_ratings_df = item_ratings_df.groupby('Username', group_keys=False).apply(lambda x: x.sample(frac=0.1))

In [ ]:
item_ratings_df = item_ratings_df.rename(columns={
    'BGGId':'item',
    'Rating':'rating',
    'Username':'user'
})

In [ ]:
user_rating_count = item_ratings_df.groupby('user').count()
user_rating_count

In [ ]:
min_rating_count = 10
user_rating_count = user_rating_count[user_rating_count['rating'] >= min_rating_count]

In [ ]:
user_list_filtered = list(user_rating_count.index)

In [ ]:
item_ratings_df_filtered = item_ratings_df[item_ratings_df['user'].isin(user_list_filtered)]

In [ ]:
item_ratings_df_filtered = item_ratings_df_filtered.drop_duplicates(subset = ['item', 'user'])

In [ ]:
item_ratings_df_filtered

### Build

In [16]:
num_recs = 10  # Number of recommendations to generate
item_item = ItemItem(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys_itemitem = Recommender.adapt(item_item)

In [17]:
recsys_itemitem.fit(item_ratings_df_filtered)

could not load LIBBLAS: Could not find module 'libblas' (or one of its dependencies). Try using the full path with constructor syntax.


In [18]:
 recsys_itemitem.recommend('zzzzzane', 10)

,item,score
0,324856,9.887308
1,275777,9.796029
2,342942,9.738921
3,249277,9.662868
4,345976,9.613461
5,341169,9.598640
6,329082,9.449887
7,313776,9.335643
8,165694,9.299688
9,284121,9.289144


In [19]:
recsys_itemitem.predict_for_user('zzzzzane', [145888, 226522])

item
145888    5.996881
226522    8.294573
dtype: float64

### Test

In [63]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [64]:
kf = KFold(n_splits=5, shuffle=True)

In [65]:
def predict(model, test_dataframe):
    real_ratings = []
    predicted_ratings = []
    
    users = test_dataframe['user'].unique()
    
    for user in users:
        current_frame = test_dataframe[test_dataframe['user'] == user]
        
        items = list(current_frame['item'])
        ratings = list(current_frame['rating'])
        predictions = list(model.predict_for_user(user, items))
        
        real_ratings += ratings
        predicted_ratings += predictions
    
    return real_ratings, predicted_ratings

In [67]:
mae_list = list()
rmse_list = list()
nans_list = list()
non_nans_list = list()

count = 1

for train_index, test_index in kf.split(item_ratings_df_filtered):
    print(f'Doing fold no: {count}')
    count += 1
    
    train_df = item_ratings_df_filtered.iloc[train_index]
    test_df = item_ratings_df_filtered.iloc[test_index]
    test_df = test_df.sample(frac = 0.001)
    
    train_df = train_df.drop_duplicates(subset = ['item', 'user'])
    test_df = test_df.drop_duplicates(subset = ['item', 'user'])
    
    print('Dataframes created')
    
    item_item = ItemItem(10, min_nbrs=3)
    recsys = Recommender.adapt(item_item)
    recsys.fit(train_df)
    print('Model is fitted')
    
    real_ratings, predicted_ratings = predict(recsys, test_df)
    
    y_test = real_ratings
    y_pred = predicted_ratings
    
    y_test_not_nan = list()
    y_pred_not_nan = list()
    
    nans = 0
    non_nans = 0
    
    for i in range(len(y_pred)):
        num = y_pred[i]
        
        if np.isnan(num):
            nans += 1
        else:
            non_nans += 1
            y_test_not_nan.append(y_test[i])
            y_pred_not_nan.append(num)
            
    print(f'nan values: {nans}')
    print(f'non-nan values: {non_nans}')
    
    mae = mean_absolute_error(y_test_not_nan, y_pred_not_nan)
    rmse = mean_squared_error(y_test_not_nan, y_pred_not_nan, squared = False)
    
    mae_list.append(mae)
    rmse_list.append(rmse)
    nans_list.append(nans)
    non_nans_list.append(non_nans)
    
# Compute average results and standard deviations
results = {
    'avg_mae' : statistics.mean(mae_list),
    'stdv_mae' : statistics.stdev(mae_list),
    'avg_rmse' : statistics.mean(rmse_list),
    'stdv_rmse' : statistics.stdev(rmse_list),
    'avg_nans': statistics.mean(nans_list),
    'avg_non_nans' : statistics.mean(non_nans_list)
}

display(results)

Doing fold no: 1
Dataframes created
Model is fitted
nan values: 1
non-nan values: 3674
Doing fold no: 2
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675
Doing fold no: 3
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675
Doing fold no: 4
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675
Doing fold no: 5
Dataframes created
Model is fitted
nan values: 0
non-nan values: 3675


{'avg_mae': 0.849650260667979,
 'stdv_mae': 0.005147369534018413,
 'avg_rmse': 1.139501925417811,
 'stdv_rmse': 0.013279363421001696,
 'avg_nans': 0.2,
 'avg_non_nans': 3674.8}

{'avg_mae': 0.849650260667979,\
 'stdv_mae': 0.005147369534018413,\
 'avg_rmse': 1.139501925417811,\
 'stdv_rmse': 0.013279363421001696,\
 'avg_nans': 0.2,\
 'avg_non_nans': 3674.8}

### Build explainable Item-based CF

In [4]:
id_to_game = item_info.set_index('BGGId')['Name'].to_dict()

NameError: name 'item_info' is not defined

In [5]:
# data = {
#     'item': [101, 102, 103, 104, 105],  # Repeating items for demonstration
#     'rating': [4, 5, 3, 4, 5],  # Repeating ratings for demonstration
#     'user': [201, 202, 203, 204, 205]  # Repeating users for demonstration
# }

# # Create a DataFrame
# fake_dataset = pd.DataFrame(data)

In [6]:
item_ratings_df_filtered = item_ratings_df_filtered.drop_duplicates(subset=['user', 'item'], keep=False)
item_ratings_df_filtered

NameError: name 'item_ratings_df_filtered' is not defined

In [7]:
# item_ratings_df_filtered = item_ratings_df_filtered.sample(frac=0.01)

In [15]:
users = item_ratings_df_filtered['user'].unique()

In [16]:
item_item = ItemItem(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys_itemitem = Recommender.adapt(item_item)

In [17]:
recsys_itemitem.fit(item_ratings_df_filtered)

could not load LIBBLAS: Could not find module 'libblas' (or one of its dependencies). Try using the full path with constructor syntax.


In [18]:
sim_matrix = item_item.sim_matrix_
sim_matrix

<CSR 21925x21925 (198881084 nnz) {
  rowptrs=[        0     16920     30477 ... 198872138 198880861 198881084]
  colinds=[ 8823  8425  2684 ... 17227 15578 13319]
  values=[1.16596048e-01 1.10199626e-01 1.08212602e-01 ... 1.27073815e-05
 6.38543095e-06 3.86370483e-06]
  dtype=float64
}>

In [19]:
sim_matrix = sim_matrix.to_scipy()

In [20]:
sim_matrix = sim_matrix.todense()

In [21]:
sim_matrix = np.array(sim_matrix)

In [22]:
item_index = item_item.item_index_

In [23]:
def get_most_similar_items(matrix, item, k, indexer):
    
    if np.isin(item, indexer):
        location = indexer.get_loc(item)
    
        arr = matrix[location]
        
        indices_of_k_largest = np.argsort(arr)[-k:][::-1]
        values_of_k_largest = arr[indices_of_k_largest]

        return indexer[indices_of_k_largest].tolist(), list(values_of_k_largest)
    else:
        return None

In [24]:
a = get_most_similar_items(sim_matrix, 50381, 2, item_index)
a

([1927, 74], [0.13583230204098787, 0.1280701897274132])

In [25]:
def recommend_for_user(user_item_df, matrix, k, indexer, user_id, threshold):
    current_df = user_item_df[user_item_df['user'] == user_id]
    current_df = current_df.sort_values('rating', ascending = False)
    
    for i in range(current_df.shape[0]):
        current_item = current_df.iloc[i]['item']
        rating = current_df.iloc[i]['rating']
        if rating >= threshold:
            recommendation = get_most_similar_items(matrix, current_item, k, indexer)
            return f"Players Who Liked \'{id_to_game[current_item]}\' Also Tried", current_item, recommendation
    
    return None

In [26]:
user_id = 'PPMP'
threshold = 8

a = recommend_for_user(item_ratings_df_filtered, sim_matrix, 5, item_index, user_id, threshold)
a

("Players Who Liked 'CATAN 3D Collector's Edition' Also Tried",
 17419,
 ([3972, 60153, 2338, 1897, 189],
  [0.13527547521496658,
   0.108278623162513,
   0.0884736537493744,
   0.0823121737970011,
   0.08150283601981548]))

### Grouped item-based CF

In [1]:
item_ratings_df_filtered

NameError: name 'item_ratings_df_filtered' is not defined

In [28]:
filename = 'clusters_ids.npy'
clusters_ids = np.load(filename, allow_pickle=True)

In [36]:
# filename = 'clusters_rand_ids.npy'
# clusters_ids = np.load(filename, allow_pickle=True)

\# of users that liked an item -> predicted item ratings (No misery + Additive) / (No misery + Most Pleasure)

In [55]:
from sklearn.metrics import ndcg_score

In [63]:
k = 5
all_best_recommendations = []
all_predicted_ratings = []

ndcg_list = []

for current_group in clusters_ids:
    
    items_list = []
    best_scores = []

    for user in current_group:
        recs = recsys_itemitem.recommend(user, 5)
        
        items = list(recs['item'])
        scores = list(recs['score'])
        
        items_list += items
        best_scores.append(scores)
    
    items_list = list(np.unique(items_list)) 
    
    predicted_ratings = []
    
    for user in current_group:
        predicted_ratings.append(list(recsys_itemitem.predict_for_user(user, items_list).values))
    
    lower_threshold = 5
    transposed_rating = np.transpose(predicted_ratings)
    filtered_items_list = []
    
    for i in range(len(transposed_rating)):
        if min(transposed_rating[i]) >= lower_threshold:
            filtered_items_list.append(items_list[i])
        
    new_predicted_ratings = []
    for user in current_group:
        new_predicted_ratings.append(list(recsys_itemitem.predict_for_user(user, filtered_items_list).values))
    
    transposed_rating = np.transpose(new_predicted_ratings)
    transposed_agg = []
    for ratings in transposed_rating:
#         transposed_agg.append(np.sum(ratings))
        transposed_agg.append(np.max(ratings))
    
    agg_to_item = dict(zip(transposed_agg, filtered_items_list))
    agg_to_item = dict(sorted(agg_to_item.items(), key=lambda x: x[0], reverse=True))
    agg_to_item = dict(list(agg_to_item.items())[:5])
    
    final_items = list(agg_to_item.values())
    predicted_ratings_list = []
    
    for user in current_group:
        predicted_ratings_list.append(list(recsys_itemitem.predict_for_user(user, final_items).values))
    
    
    current_ndcg = ndcg_score(best_scores, predicted_ratings_list)
    ndcg_list.append(current_ndcg)

In [64]:
np.mean(ndcg_list)

0.9966201795572952

Addition\
sim: 0.9968280467337415\
\
MP\
sim: 0.9966201795572952